In [1]:
"""
Association rule mining
"""
import os
import numpy as np
import pandas as pd

In [2]:
path = os.getcwd()
files = os.listdir(path)

In [3]:
"""

Key ideas to report in the paper
1. General statistics: Number of pipes with defects, most frequent defects 
2. Number of instances of two defects being within the same vicinity, probabilities of these occurrences
3. Support, lift, etc.

"""

'\n\nKey ideas to report in the paper\n1. General statistics: Number of pipes with defects, most frequent defects \n2. Number of instances of two defects being within the same vicinity, probabilities of these occurrences\n3. Support, lift, etc.\n\n'

In [12]:
"""
Helper functions
"""

def change_defect_names(df_cond):
    """ 
    Change the names of defects: e.g., FL => Fracture, TB => Tap
    Takes df_cond as input. 
    """
    deposit_codes = ['DAE', 'DAGS', 'DAR', 'DAZ', 'DSV', 'DSGV', 'DSC', 'DSZ', 'DNF', 'DNGV', 'DNZ']
    deformed_codes = ['DR', 'DFBR', 'DFBI', 'DFC', 'DFE', 'DTBR', 'DTBI']
    infiltration_codes = ['IS', 'ISB', 'ISJ', 'ISC', 'ISL', 'IW', 'IWB', 'IWC', 'IWJ', 'IWL', 'ID', 'IDB', 'IDC', 'IDJ', 'IDL', 'IR', 'IRB', 'IRC', 'IRJ', 'IRL', 'IG', 'IGB', 'IGC', 'IGL', 'IGJ' ]
    hole_codes = ['HSV', 'HVV']
    fracture_codes = ['FL', 'FC', 'FM', 'FS', 'FH']
    crack_codes = ['CL', 'CC', 'CM', 'CS', 'CH']
    broken_codes = ['BSV', 'BVV']
    collapse_codes = ['X']
    
    df_cond.loc[df_cond['PACP_Code'].isin(deposit_codes), 'PACP_Code'] = 'Deposit'
    df_cond.loc[df_cond['PACP_Code'].isin(deformed_codes), 'PACP_Code'] = 'Deformed'
    df_cond.loc[df_cond['PACP_Code'].isin(infiltration_codes), 'PACP_Code'] = 'Infiltration'
    df_cond.loc[df_cond['PACP_Code'].isin(hole_codes), 'PACP_Code'] = 'Hole'
    df_cond.loc[df_cond['PACP_Code'].isin(fracture_codes), 'PACP_Code'] = 'Fracture'
    df_cond.loc[df_cond['PACP_Code'].isin(crack_codes), 'PACP_Code'] = 'Crack'
    df_cond.loc[df_cond['PACP_Code'].isin(broken_codes), 'PACP_Code'] = 'Broken'
    df_cond.loc[df_cond['PACP_Code'].isin(collapse_codes), 'PACP_Code'] = 'Collapse'
    
    df_cond.loc[df_cond['PACP_Code'].str.startswith('T'), 'PACP_Code'] = 'Tap'
    df_cond.loc[df_cond['PACP_Code'].str.startswith('A'), 'PACP_Code'] = 'Manhole'
    df_cond.loc[df_cond['PACP_Code'].str.startswith('J'), 'PACP_Code'] = 'Joint Offset'
    df_cond.loc[df_cond['PACP_Code'].str.startswith('R'), 'PACP_Code'] = 'Root'
    
    return df_cond

def delete_rows(df_cond, keep_defects):
    """
    Delete rows where the defects that we want to consider are not present
    """
    df_cond = df_cond[df_cond['PACP_Code'].isin(keep_defects)]
    return df_cond
    

def select_df(df, insp_id, defect_list):
    """
    Helper function used by count_defect_pairs()
    This function only keeps the rows where ['defect1', 'defect2'] are present
    It helps speed up the computation
    """
    df = df[(df['InspectionID'] == insp_id)]
    return  df[df['PACP_Code'].isin(defect_list)]

def delete_inspections_with_no_defects(df_cond, defects):
    """
    Delete those inspections where there aren't any defects at all. 
    By defects we mean fractures, cracks, etc. Not tap and manhole
    """
    insp_ids = np.unique(list(df_cond['InspectionID']))

    keep_ids = [] # Inspection IDs that we want to keep

    for insp_id in insp_ids:
        df_temp = df_cond[(df_cond['InspectionID'] == insp_id)]
        if df_temp['PACP_Code'].isin(defects).sum() > 0:
            keep_ids.append(insp_id)
    
    return df_cond[df_cond['InspectionID'].isin(keep_ids)]

def count_defect_pairs(df_cond, keep_defects, distance_threshold):
    """
    Count the number of defect pairs that are < distance_threshold within each other
    """
    defect_pair_counts = []
    
    for a in range(0, len(keep_defects)): 
        for b in range(a+1, len(keep_defects)):

            pair_count = 0
            defect_pair = [keep_defects[a], keep_defects[b]] # Pair of defects whose no. of occurrences we wish to calculate

            for insp_id in df_cond['InspectionID'].unique():
            
            # Inspection id which we are counting for
                df = select_df(df_cond, insp_id, defect_pair)
                insp_id = list(df['InspectionID'])
                dist = list(df['Distance'])
                defects = list(df['PACP_Code']) # Defects which are selected from a particular inspection

                for i in range (0, len(defects) - 1):
                    for j in range (i+1, len(defects)):
                        defect_1, defect_2 = defects[i], defects[j]
                        if (defect_1 == defect_pair[0] and defect_2 == defect_pair[1]) or (defect_2 == defect_pair[0] and defect_1 == defect_pair[1]):
                            if abs(dist[i] - dist[j]) <= distance_threshold:
                                pair_count = pair_count + 1

            defect_pair_counts.append([keep_defects[a], keep_defects[b], pair_count])
            
    return defect_pair_counts


   

def create_econometric_database(df_insp, cols, defects):
    df_insp = df_insp[cols]

    f, c, t, m, j, r, d, b, h, cp = [], [], [], [], [], [], [], [], [], []

    insp_ids = list(df_insp['InspectionID'])

    for insp_id in insp_ids:

        df_temp = df_cond.loc[df_cond['InspectionID'] == insp_id]

        f.append(len(df_temp[df_temp['PACP_Code'] == 'Fracture']))
        c.append(len(df_temp[df_temp['PACP_Code'] == 'Crack']))
        t.append(len(df_temp[df_temp['PACP_Code'] == 'Tap']))
        m.append(len(df_temp[df_temp['PACP_Code'] == 'Manhole']))
        j.append(len(df_temp[df_temp['PACP_Code'] == 'Joint Offset']))
        r.append(len(df_temp[df_temp['PACP_Code'] == 'Root']))
        d.append(len(df_temp[df_temp['PACP_Code'] == 'Deposit']))
        b.append(len(df_temp[df_temp['PACP_Code'] == 'Broken']))
        h.append(len(df_temp[df_temp['PACP_Code'] == 'Hole']))
        cp.append(len(df_temp[df_temp['PACP_Code'] == 'Collapse']))

    df_insp['Fracture'] = f
    df_insp['Crack'] = c
    df_insp['Tap'] = t
    df_insp['Manhole'] = m
    df_insp['Joint Offset'] = j
    df_insp['Root'] = r
    df_insp['Deposit'] = d
    df_insp['Broken'] = b
    df_insp['Hole'] = h
    df_insp['Collapse'] = cp
    
    return df_insp

In [13]:
cluster_size = 10

# Read the database of condition information from CSV
df_cond = pd.read_csv('Conditions_Taylors.csv', sep=',')
df_cond.head();
print("Total number of inspections to begin with are: {}".format(df_cond['InspectionID'].max()))

# Change names of defects
df_cond = change_defect_names(df_cond)

# Keep only the rows which contain these defects. Delete all else
keep_defects_struct = ['Fracture', 'Crack', 'Joint Offset', 'Broken', 'Hole', 'Collapse']
keep_defects_operat = ['Root', 'Deposit']

df_cond = delete_rows(df_cond, keep_defects_struct)

df_cond = delete_inspections_with_no_defects(df_cond, keep_defects_struct)
print("Length : {}".format(calculate_length_of_pipeline(df_cond)))


# Count the number of inspections that remain after the deletion
print("\nNumber of inspections which contain defects is: {}".format(df_cond['InspectionID'].nunique()))

# List the number of defects
print("\nNumber of defects is as follows: \n{}".format(df_cond['PACP_Code'].value_counts()))

"""
Counting only Fracture and Tap
Delete all inspections which do not have both Fracture and Tap
"""

keep_defects = ['Joint Offset', 'Fracture']

# Deletes those inspections which don't have defect1 or defect2
df_cond = delete_inspections_with_no_defects(df_cond, [keep_defects[0], keep_defects[1]])

# Count the number of inspections that remain after the deletion
print("\nNumber of inspections with {} and {} is: {}".format(keep_defects[0], keep_defects[1], df_cond['InspectionID'].nunique()))

# List the number of defects
print("\nNumber of defects in these inspections is as follows: \n{}".format(df_cond['PACP_Code'].value_counts()))

# Counting the number of defect clusters within 5 feet
defect_pair_counts = count_defect_pairs(df_cond, keep_defects, cluster_size)

print('\nNumber of ({}, {}) clusters is: {}'.format(keep_defects[0], keep_defects[1], defect_pair_counts[0][2]))

# Count the total length of pipe in this dataframe
length = calculate_length_of_pipeline(df_cond)

print("Length : {}".format(length))

Total number of inspections to begin with are: 3618
Length : 108581.10015941903

Number of inspections which contain defects is: 768

Number of defects is as follows: 
Crack           789
Fracture        354
Broken          172
Joint Offset    144
Hole             34
Name: PACP_Code, dtype: int64

Number of inspections with Joint Offset and Fracture is: 352

Number of defects in these inspections is as follows: 
Fracture        354
Crack           225
Joint Offset    144
Broken           64
Hole             10
Name: PACP_Code, dtype: int64

Number of (Joint Offset, Fracture) clusters is: 6
Length : 52995.300032654


In [6]:
num_defect1 = df_cond['PACP_Code'].value_counts()[keep_defects[0]]
num_defect2 = df_cond['PACP_Code'].value_counts()[keep_defects[1]]
print("Number of {} is {}".format(keep_defects[0], num_defect1))
print("Number of {} is {}".format(keep_defects[1], num_defect2))

spacing1 = length/num_defect1
spacing2 = length/num_defect2

print("Length: {}".format(length))
print("Average {} spacing: {} feet".format(keep_defects[0], spacing1))
print("Average {} spacing: {} feet".format(keep_defects[1], spacing2))

Number of Joint Offset is 1943
Number of Fracture is 8558
Length: 419601.8005263697
Average Joint Offset spacing: 215.955635885934 feet
Average Fracture spacing: 49.03035762168377 feet


In [7]:
# If Taps and Fractures were uniformly distributed, calculate the number of co-occurrences
import numpy as np

defect1 = [[keep_defects[0], i, 1] for i in np.arange(0, length, spacing1)]
defect2 = [[keep_defects[1], i, 1] for i in np.arange(0, length, spacing2)]
df_cond_fake = pd.DataFrame(defect1+defect2, columns = ['PACP_Code', 'Distance', 'InspectionID'])
defect_pair_counts = count_defect_pairs(df_cond_fake, [keep_defects[0], keep_defects[1]], cluster_size)
print('\nNumber of ({}, {}) clusters is: {}'.format(keep_defects[0], keep_defects[1], defect_pair_counts[0][2]))


Number of (Joint Offset, Fracture) clusters is: 793


In [14]:
"""
Count number of defect hotspots

TODO: Compare with uiform distribution/Weibull distribution/Try fitting curve

"""

'\nCount number of defect hotspots\n\nTODO: Compare with uiform distribution/Weibull distribution/Try fitting curve\n\n'

In [21]:
# Helper functions

def delete_rows(df_cond, keep_defects):
    """
    Delete rows where the defects that we want to consider are not present
    """
    df_cond = df_cond[df_cond['PACP_Code'].isin(keep_defects)]
    return df_cond
    

def select_df(df, insp_id, defect_list):
    """
    Helper function used by count_defect_pairs()
    This function only keeps the rows where ['defect1', 'defect2'] are present
    It helps speed up the computation
    """
    df = df[(df['InspectionID'] == insp_id)]
    return  df[df['PACP_Code'].isin(defect_list)]

In [22]:
# Defect Codes

deposit_codes = ['DAE', 'DAGS', 'DAR', 'DAZ', 'DSV', 'DSGV', 'DSC', 'DSZ', 'DNF', 'DNGV', 'DNZ']
deformed_codes = ['DR', 'DFBR', 'DFBI', 'DFC', 'DFE', 'DTBR', 'DTBI']
infiltration_codes = ['IS', 'ISB', 'ISJ', 'ISC', 'ISL', 'IW', 'IWB', 'IWC', 'IWJ', 'IWL', 'ID', 'IDB', 'IDC', 'IDJ', 'IDL', 'IR', 'IRB', 'IRC', 'IRJ', 'IRL', 'IG', 'IGB', 'IGC', 'IGL', 'IGJ' ]
hole_codes = ['HSV', 'HVV']
fracture_codes = ['FL', 'FC', 'FM', 'FS', 'FH', 'FH2', 'FH3', 'FH4']
crack_codes = ['CL', 'CC', 'CM', 'CS', 'CH', 'CH2', 'CH3', 'CH4']
broken_codes = ['BSV', 'BVV']
collapse_codes = ['X']

tap_codes = ['TB', 'TBI', 'TBD', 'TBC', 'TBA', 'TF', 'TFI', 'TFD', 'TFC', 'TFA', 'TFB', 'TR', 'TRI', 'TRD', 'TRC', 'TRA', 'TRB', 'TS', 'TSI', 'TSD', 'TSA', 'TSB']
root_codes = ['RFB', 'RFL', 'RFC', 'RFJ', 'RMB', 'RML', 'RMC', 'RMJ', 'RBB', 'RBL', 'RBC', 'RBJ', 'RTB', 'RTL', 'RTC', 'RTJ']
joint_offset_codes = ['JOS', 'JOM', 'JOL', 'JOSD', 'JOMD', 'JOLD', 'JSS', 'JSM', 'JSL', 'JAS', 'JAM', 'JAL']

defects_all = deposit_codes+deformed_codes+infiltration_codes+hole_codes+fracture_codes+crack_codes+broken_codes+root_codes+joint_offset_codes+collapse_codes
defects_struct = deformed_codes+hole_codes+fracture_codes+crack_codes+broken_codes+joint_offset_codes+collapse_codes
defects_operat = root_codes + deposit_codes

keep_defects = defects_struct

In [38]:
# Count number of defect zones
df_cond = pd.read_csv('Conditions_Combined.csv', sep=',')
df_cond.head();
print("Total number of inspections to begin with are: {}".format(df_cond['InspectionID'].nunique()))

# Change names of defects
# df_cond = change_defect_names(df_cond)

# Keep only the rows which contain these defects. Delete all else
# keep_defects = ['Fracture', 'Crack', 'Joint Offset', 'Root', 'Deposit', 'Broken', 'Hole', 'Infiltration', 'Deformed']


df_cond = delete_rows(df_cond, keep_defects)

# Delete inspections which have no 'defects', i.e, fracture, crack, root, joint offset, deposit, broken
df_cond = delete_inspections_with_no_defects(df_cond, keep_defects)

print("Number of inspections after deletion are: {}".format(df_cond['InspectionID'].nunique()))

Total number of inspections to begin with are: 8891
Number of inspections after deletion are: 2825


In [41]:
# List the number of defects
df_counts = df_cond['PACP_Code'].value_counts()
print("\nNumber of defects in these inspections is as follows: \n{}".format(df_counts))
df_counts.to_csv('defect_counts.csv')

# Count the total length of pipe in this dataframe
length = calculate_length_of_pipeline(df_cond)
print("Length : {}".format(length))


Number of defects in these inspections is as follows: 
FM     4268
CL     3321
CM     3211
CC     1900
JOM    1642
FS     1519
FC     1429
FL     1342
CS      765
BSV     320
HSV     310
JOL     189
BVV     138
HVV     128
JSM      72
FH3      27
FH4      26
JSL      19
JAM      18
CH3      14
FH2      14
CH2      12
CH4      10
JAL       3
Name: PACP_Code, dtype: int64
Length : 441077.6999943439


In [78]:
thresh = 3
insps = df_cond['InspectionID'].unique()
zones = []

for insp in insps:
    
    df_temp = df_cond[df_cond['InspectionID'] == insp]
    df_temp = df_temp.sort_values(by=["Distance"])
    indices = df_temp.index
    defect_prev, defect_curr = "",""
    dist_prev, dist_curr = 0, 0
    zone_curr = []
    
    for index in indices:
        defect_curr = df_temp.at[index, 'PACP_Code'] # Defect code at current index
        dist_curr = float(df_temp.at[index, 'Distance']) # Distance of defect at current index        
    
        if abs(dist_curr - dist_prev) < thresh:
            zone_curr.append((insp, defect_curr, dist_curr))
        else:           
            zones.append(zone_curr)
            zone_curr = []
            zone_curr.append((insp, defect_curr, dist_curr))
                        
        dist_prev = dist_curr
        defect_prev = defect_curr
        
    zones.append(zone_curr)

In [79]:
len(zones)
zones[0:10]

[[],
 [(1, 'JOL', 11.10000038)],
 [],
 [(6, 'FM', 21.10000038)],
 [(6, 'FL', 75.5)],
 [],
 [(24, 'FL', 101.4000015)],
 [],
 [(25, 'CC', 13.69999981)],
 [(25, 'CL', 19.5)]]

In [80]:
# Delete empty zones
zones = list(filter(lambda a: a!=[], zones))

In [81]:
"""
Calculate number of colocated defects
"""

colocated = 0
total = 0
max_colocated = 0
num_def_in_cluster = 5
filtered_zones = []

for zone in zones:
    if len(zone) >= num_def_in_cluster:
        colocated += len(zone)
        filtered_zones.append(zone)
        if len(zone) > max_colocated:
            max_colocated = len(zone)
            max_zone = zone
    total += len(zone)
    
print(f'Number of defects in clusters with min {num_def_in_cluster} defects is {colocated}')
print(f'Total number of defects is {total}')
print(f'Max defects in a zone is {max_colocated}')
print(f'Number of clusters {len(filtered_zones)}')

Number of defects in clusters with min 5 defects is 1006
Total number of defects is 20697
Max defects in a zone is 16
Number of clusters 167


In [82]:
max_zone

[(1983, 'FM', 23.70000076),
 (1983, 'BSV', 26.29999924),
 (1983, 'FS', 29.0),
 (1983, 'BSV', 31.29999924),
 (1983, 'BSV', 34.0),
 (1983, 'FM', 36.59999847),
 (1983, 'BSV', 39.20000076),
 (1983, 'BSV', 41.70000076),
 (1983, 'BSV', 44.29999924),
 (1983, 'FM', 44.29999924),
 (1983, 'BSV', 46.90000153),
 (1983, 'BSV', 46.90000153),
 (1983, 'FM', 46.90000153),
 (1983, 'BSV', 49.59999847),
 (1983, 'BSV', 52.09999847),
 (1983, 'FM', 52.09999847)]

In [87]:
# Count number of zones which have fractures or cracks only
num_zones_with_major_defects = 0


for zone in filtered_zones:
    for _, defect_code, _ in zone:
        if defect_code in broken_codes + deformed_codes + hole_codes:
            num_zones_with_major_defects += 1
            break

print(num_zones_with_major_defects)

54


In [92]:
filtered_zones

import csv

with open("filered_zones.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(filtered_zones)

In [50]:
"""
Calculate the number of zones of various lengths
"""

num_zones = {i:0 for i in range(1, max_colocated + 1)}
for zone in zones:
    if len(zone) >= 1:
        num_zones[len(zone)] += 1
        
num_zones

{1: 13318,
 2: 2151,
 3: 481,
 4: 157,
 5: 88,
 6: 47,
 7: 10,
 8: 10,
 9: 4,
 10: 2,
 11: 2,
 12: 2,
 13: 0,
 14: 0,
 15: 0,
 16: 2}

In [89]:
"""
Count the number of zones with more than 3 fractures
"""
hotspots = []
count_hotspots = 0
for zone in zones:
    count_def = 0
    if len(zone) >= 5:
        for _, pacp_code, _ in zone:
            if pacp_code in fracture_codes:
                count_def += 1
        if count_def > 3:
            count_hotspots += 1
            hotspots.append(zone)

hotspots

[[(1027, 'FM', 114.3000031),
  (1027, 'FM', 116.09999850000001),
  (1027, 'JOM', 117.90000149999999),
  (1027, 'FM', 119.40000149999999),
  (1027, 'FM', 122.09999850000001)],
 [(1713, 'FM', 228.89999390000003),
  (1713, 'FM', 231.6999969),
  (1713, 'FC', 234.0),
  (1713, 'FC', 235.1999969),
  (1713, 'FM', 236.89999390000003)],
 [(1763, 'FM', 46.5),
  (1763, 'FM', 47.59999847),
  (1763, 'FL', 49.59999847),
  (1763, 'FM', 52.20000076),
  (1763, 'FM', 54.79999924)],
 [(1899, 'FS', 180.3999939),
  (1899, 'FS', 183.0),
  (1899, 'CL', 183.6999969),
  (1899, 'FM', 185.3000031),
  (1899, 'CL', 186.1999969),
  (1899, 'HSV', 188.1000061),
  (1899, 'FS', 188.3000031)],
 [(1942, 'CL', 83.40000153),
  (1942, 'FL', 85.80000305),
  (1942, 'FL', 86.30000305),
  (1942, 'FS', 88.30000305),
  (1942, 'FS', 91.0)],
 [(1983, 'FM', 23.70000076),
  (1983, 'BSV', 26.29999924),
  (1983, 'FS', 29.0),
  (1983, 'BSV', 31.29999924),
  (1983, 'BSV', 34.0),
  (1983, 'FM', 36.59999847),
  (1983, 'BSV', 39.20000076),
 

{1: 16693,
 2: 4132,
 3: 1354,
 4: 613,
 5: 293,
 6: 149,
 7: 106,
 8: 78,
 9: 52,
 10: 38,
 11: 18,
 12: 20,
 13: 10,
 14: 15,
 15: 6,
 16: 9,
 17: 0,
 18: 1,
 19: 2,
 20: 4,
 21: 3,
 22: 0,
 23: 6,
 24: 4,
 25: 0,
 26: 1,
 27: 0,
 28: 0,
 29: 2}

In [90]:
"""
Most frequent defect in colocated zone
"""
defects = {defect_code:0 for defect_code in keep_defects}
for zone in zones:
    if len(zone) >= 3:
        for defect in zone:
            defects[defect[1]] += 1 

In [91]:
defects

{'DR': 0,
 'DFBR': 0,
 'DFBI': 0,
 'DFC': 0,
 'DFE': 0,
 'DTBR': 0,
 'DTBI': 0,
 'HSV': 146,
 'HVV': 42,
 'FL': 298,
 'FC': 101,
 'FM': 561,
 'FS': 327,
 'FH': 0,
 'FH2': 2,
 'FH3': 10,
 'FH4': 14,
 'CL': 530,
 'CC': 116,
 'CM': 411,
 'CS': 94,
 'CH': 0,
 'CH2': 5,
 'CH3': 6,
 'CH4': 6,
 'BSV': 119,
 'BVV': 11,
 'JOS': 0,
 'JOM': 209,
 'JOL': 55,
 'JOSD': 0,
 'JOMD': 0,
 'JOLD': 0,
 'JSS': 0,
 'JSM': 8,
 'JSL': 6,
 'JAS': 0,
 'JAM': 0,
 'JAL': 0,
 'X': 0}

In [131]:
"""
Finding severity of defect clusters and finding length of these clusters.
Will allow for the best patch repair

"""

'\nFinding severity of defect clusters and finding length of these clusters.\nWill allow for the best patch repair\n\n'

In [55]:
"""
Interdefect association rule mining
"""

'\nInterdefect association rule mining\n'

In [68]:
# Create matrix: https://towardsdatascience.com/association-rule-mining-be4122fc1793
import numpy as np

co_matrix = []

for zone in zones:
    row = np.zeros(len(keep_defects))
    for _, defect, _ in zone:
        defect_index = keep_defects.index(defect)
        row[defect_index] = 1
    co_matrix.append(row)

In [69]:
zones[:5]

[[(1, 'JOL', 11.10000038)],
 [(6, 'FM', 21.10000038), (6, 'FL', 75.5)],
 [(24, 'FL', 101.4000015)],
 [(25, 'CC', 13.69999981),
  (25, 'CL', 19.5),
  (25, 'CC', 30.70000076),
  (25, 'CM', 40.29999924),
  (25, 'CM', 50.20000076),
  (25, 'FL', 64.90000153)],
 [(27, 'CM', 98.69999695), (27, 'FC', 113.5), (27, 'CL', 192.1000061)]]

In [70]:
len(co_matrix)

2825

In [71]:
co_matrix[:2]

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.])]

In [72]:
len(co_matrix[0]) == len(keep_defects)

True

In [73]:
def count_pairs(x, y, keep_defects, co_matrix):
    count_x, count_y, count_xy = 0, 0, 0
    for row in co_matrix:
        if row[x] == 1 and row[y] == 1.0:
            count_xy += 1
        if row[x] == 1.0:
            count_x += 1
        if row[y] == 1.0:
            count_y += 1
            
    return count_x, count_y, count_xy

In [74]:
def support_confidence_xy(x, y, keep_defects, co_matrix):
    count_x, count_y, count_xy = count_pairs(x, y, keep_defects, co_matrix)
    support_xy = count_xy/len(co_matrix)
    support_x = count_x/len(co_matrix)
    support_y = count_y/len(co_matrix)
    if count_x ==0 or count_y == 0 or count_xy ==0:
        confidence_xy = 0
    else:
        confidence_xy = count_xy/count_x
    return support_x, support_y, support_xy, confidence_xy

In [75]:
min_support = 0.01
min_confidence = 0.1
rules = []

for x in range(0, len(keep_defects)):
    for y in range(0, len(keep_defects)):
        if x!=y:
            support_x, support_y, support_xy, confidence_xy = support_confidence_xy(x, y, keep_defects, co_matrix)
            if support_xy > min_support and confidence_xy > min_confidence:
                lift_xy = support_xy/(support_x*support_y)
                rules.append([keep_defects[x], keep_defects[y], support_xy, confidence_xy, lift_xy])


In [76]:
rules.sort(key=lambda tup: tup[3], reverse=True)  # sorts in place by confidence
rules

[['FL', 'FM', 0.15008849557522125, 0.7464788732394366, 1.8242238900531218],
 ['HSV', 'FM', 0.04106194690265487, 0.7204968944099379, 1.7607298673945282],
 ['FS', 'FM', 0.12176991150442477, 0.7034764826175869, 1.71913586798848],
 ['HVV', 'FM', 0.018761061946902656, 0.6463414634146342, 1.579510929192337],
 ['HVV', 'CL', 0.018053097345132742, 0.6219512195121951, 1.637476416702657],
 ['BSV', 'FM', 0.04176991150442478, 0.6178010471204188, 1.5097646696498126],
 ['CS', 'FM', 0.0775221238938053, 0.6151685393258427, 1.5033314217954201],
 ['CM', 'CL', 0.1872566371681416, 0.6101499423298731, 1.6064059525460312],
 ['FL', 'CL', 0.12035398230088495, 0.5985915492957746, 1.5759749550424635],
 ['JOL', 'JOM', 0.021946902654867255, 0.5961538461538461, 2.783693579148124],
 ['CS', 'CL', 0.07469026548672567, 0.5926966292134831, 1.5604547786841474],
 ['CS', 'FS', 0.07362831858407079, 0.5842696629213483, 3.3753819994944974],
 ['FC', 'FM', 0.1561061946902655, 0.5719844357976653, 1.3977993348861633],
 ['FS', 'CL

In [143]:
""" 

Interesting Rules from Association Rules on Pipe-Level 

['JOL', 'FM', 0.011782032400589101, 0.5384615384615384, 2.213933989885547]
['RFB', 'FM', 0.014727540500736377, 0.6542056074766355, 2.689826342851599],
['RFB', 'CL', 0.011782032400589101, 0.5233644859813084, 2.318314447221956],

Interesting Rules from Association Rules on Cluster-Level (Threshold = 20) 
['JOM', 'FM', 0.019923103809856693, 0.24332977588046958, 1.5513459360312503],
['JOM', 'CL', 0.01896190143306536, 0.23159018143009605, 1.6111355843684008]
['JOM', 'CM', 0.01494232785739252, 0.1824973319103522, 1.4917853331300504]]

Interesting Rules from Association Rules on Cluster-Level (Threshold = 30) 
['HSV', 'FM', 0.01151047887935715, 0.6198830409356725, 3.8209524256871545],
['BSV', 'FM', 0.01205342599630796, 0.5522388059701493, 3.40399408579592],
['JOM', 'FM', 0.0253013356499077, 0.2757396449704142, 1.6996562185626136],
['JOM', 'CL', 0.023672494299055272, 0.2579881656804734, 1.7291215558598831],
['CS', 'JOM', 0.011619068302747313, 0.25235849056603776, 2.7502595735179187],
['JOM', 'CM', 0.021609295254642197, 0.23550295857988165, 1.9040796712573573]
['FS', 'JOM', 0.013465088500380064, 0.20261437908496732, 2.2081370615307265]


"""

" \n\nInteresting Rules from Association Rules on Pipe-Level \n\n['JOL', 'FM', 0.011782032400589101, 0.5384615384615384, 2.213933989885547]\n['RFB', 'FM', 0.014727540500736377, 0.6542056074766355, 2.689826342851599],\n['RFB', 'CL', 0.011782032400589101, 0.5233644859813084, 2.318314447221956],\n\nInteresting Rules from Association Rules on Cluster-Level (Threshold = 20) \n['JOM', 'FM', 0.019923103809856693, 0.24332977588046958, 1.5513459360312503],\n['JOM', 'CL', 0.01896190143306536, 0.23159018143009605, 1.6111355843684008]\n['JOM', 'CM', 0.01494232785739252, 0.1824973319103522, 1.4917853331300504]]\n\nInteresting Rules from Association Rules on Cluster-Level (Threshold = 30) \n['HSV', 'FM', 0.01151047887935715, 0.6198830409356725, 3.8209524256871545],\n['BSV', 'FM', 0.01205342599630796, 0.5522388059701493, 3.40399408579592],\n['JOM', 'FM', 0.0253013356499077, 0.2757396449704142, 1.6996562185626136],\n['JOM', 'CL', 0.023672494299055272, 0.2579881656804734, 1.7291215558598831],\n['CS', '

In [78]:
num_rows = 0
for row in co_matrix:
    if np.count_nonzero(row == 1) > 1:
        num_rows += 1
        
print(f'Number of rows with more than 1 defect is {num_rows}')

Number of rows with more than 1 defect is 3995


In [102]:
lift = support/(support_x*support_y)
lift

1.0530775826485135

In [181]:
support

0.07973102785782901

In [36]:
"""
Make a database with external factors for econometric analysis
"""
# Read the inspections database
df_insp = pd.read_csv('Inspections_Hazen_Sawyer.csv')

# Columns that we want to keep
cols = ['InspectionID', 'Height', 'Down_Rim_to_Invert', 'Up_Rim_to_Invert', 'Height', 'Total_Length', 'Location_Code', 'Location_Details', 'Material', 'Weather', 'Street']

# Defect pairs to keep
keep_defects = ['Fracture', 'Crack', 'Tap', 'Manhole', 'Joint Offset', 'Root', 'Deposit', 'Broken']

df_insp = create_econometric_database(df_insp, cols, keep_defects)

df_insp.head()

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-